# Example of stream cipher applied to audio files

In [1]:
!pip install scipy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import numpy as np
from scipy.io import wavfile
from IPython.display import Audio

# --- Step 1: Load audio ---
rate, data = wavfile.read("tres.wav")  # must be PCM WAV
print("Sample rate:", rate)
print("Shape:", data.shape, "dtype:", data.dtype)

# Convert stereo to mono if needed
if data.ndim > 1:
    data = data[:,0]  

# Flatten to 16-bit integers
plaintext = data.astype(np.int16)

# --- Step 2: Convert to bytes ---
plaintext_bytes = plaintext.tobytes()

# --- Step 3: Define LCG keystream generator ---
def lcg(seed, a=1664525, c=1013904223, m=2**32, nbytes=0):
    """Generate keystream bytes using an LCG."""
    x = seed
    out = []
    for _ in range(nbytes):
        x = (a * x + c) % m
        out.append((x >> 16) & 0xFF)  # take one byte from state
    return bytes(out)

# --- Step 4: Encrypt ---
seed = 12345
keystream = lcg(seed, nbytes=len(plaintext_bytes))
ciphertext_bytes = bytes([p ^ k for p, k in zip(plaintext_bytes, keystream)])

# --- Step 5: Decrypt (same seed!) ---
keystream2 = lcg(seed, nbytes=len(ciphertext_bytes))
decrypted_bytes = bytes([c ^ k for c, k in zip(ciphertext_bytes, keystream2)])
decrypted_data = np.frombuffer(decrypted_bytes, dtype=np.int16)

# --- Step 6: Save results ---
wavfile.write("encrypted.wav", rate, np.frombuffer(ciphertext_bytes, dtype=np.int16))
wavfile.write("decrypted.wav", rate, decrypted_data)

print("Files saved: encrypted.wav (noisy sound), decrypted.wav (original).")


Sample rate: 8000
Shape: (320000,) dtype: int16
Files saved: encrypted.wav (noisy sound), decrypted.wav (original).


In [3]:
Audio("decrypted.wav")  

The text of the poem, translated into English, is as follows (it is a poem by <strong>Ernesto Cardenal</strong>, a well-known poet from Nicaragua):

<strong> LIKE EMPTY BEER CANS </strong>

My days have been like empty beer cans

and stubbed-out cigarette ends.

My life has passed me by like the figures who appear

and disappear on a television screen.

Like cars passing by at speed along the roads

with girls laughing and music from the radio . . .

And beauty was as transient as the models of those cars

and the fleeting hits that blasted from the radios

and were forgotten.

And nothing is left of those days,

nothing, besides the empty cans and stubbed-out dog-ends,

smiles on washed-out photos, torn coupons,

and the sawdust with which, at dawn,

they swept out the bars.

<font color='blue'> The original Spanish poem is: </font> 
    
<strong> COMO LATAS DE CERVEZA VACÍAS</strong> 

Como latas de cerveza vacías y colillas

de cigarrillos apagados, han sido mis días.

Como figuras que pasan por una pantalla de televisión

y desaparecen, así ha pasado mi vida.

Como los automóviles que pasaban rápidos por las carreteras

con risas de muchachas y música de radios...

Y la belleza pasó rápida, como el modelo de los autos

y las canciones de los radios que pasaron de moda.

Y no ha quedado nada de aquellos días, nada

más que latas vacías y colillas apagadas,

risas en fotos marchitas, boletos rotos,

y el aserrín con que al amanecer barrieron los bares.

In [4]:
Audio("encrypted.wav")  